In [1]:
from ultralytics.nn.modules import (
    AIFI,
    C1,
    C2,
    C2PSA,
    C3,
    C3TR,
    ELAN1,
    OBB,
    PSA,
    SPP,
    SPPELAN,
    # SPPF,
    # A2C2f,
    AConv,
    ADown,
    # Bottleneck,
    BottleneckCSP,
    # C2f,
    C2fAttn,
    C2fCIB,
    C2fPSA,
    C3Ghost,
    C3k2,
    C3x,
    CBFuse,
    CBLinear,
    Classify,
    Concat,
    # Conv,
    Conv2,
    ConvTranspose,
    Detect,
    DWConv,
    DWConvTranspose2d,
    Focus,
    GhostBottleneck,
    GhostConv,
    HGBlock,
    HGStem,
    ImagePoolingAttn,
    # Index,
    # LRPCHead,
    Pose,
    RepC3,
    RepConv,
    RepNCSPELAN4,
    RepVGGDW,
    ResNetLayer,
    RTDETRDecoder,
    SCDown,
    Segment,
    # TorchVision,
    WorldDetect,
    # YOLOEDetect,
    # YOLOESegment,
    v10Detect,
)

In [2]:
import model_classes
print(dir(model_classes))

['Bottleneck', 'C2f', 'Concat', 'Conv', 'DFL', 'Head', 'SPPF', 'Upsample', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'cv2', 'math', 'nn', 'np', 'os', 'random', 'torch', 'yolo_params']


In [3]:
from model_classes import(
    Conv,
    C2f,
    SPPF,
    Upsample,
    DFL,
    Head,
    # meow_test,
    Bottleneck,
    Concat,
)

In [4]:
from ultralytics.utils import DEFAULT_CFG_DICT, DEFAULT_CFG_KEYS, LOGGER, yaml_load as YAML, colorstr, emojis
from ultralytics.utils.checks import check_requirements, check_suffix, check_yaml
import torch
import torch.nn as n
import contextlib
from ultralytics.utils.ops import make_divisible
from copy import deepcopy
from pathlib import Path
import re

In [5]:
def guess_model_scale(model_path):
    """
    Extract the size character n, s, m, l, or x of the model's scale from the model path.

    Args:
        model_path (str | Path): The path to the YOLO model's YAML file.

    Returns:
        (str): The size character of the model's scale (n, s, m, l, or x).
    """
    try:
        return re.search(r"yolo(e-)?[v]?\d+([nslmx])", Path(model_path).stem).group(2)  # noqa
    except AttributeError:
        return ""

In [6]:
import ultralytics
print(ultralytics.__file__)


c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\ultralytics\__init__.py


In [7]:
def yaml_model_load(path):
    """
    Load a YOLOv8 model from a YAML file.

    Args:
        path (str | Path): Path to the YAML file.

    Returns:
        (dict): Model dictionary.
    """
    path = Path(path)
    if path.stem in (f"yolov{d}{x}6" for x in "nsmlx" for d in (5, 8)):
        new_stem = re.sub(r"(\d+)([nslmx])6(.+)?$", r"\1\2-p6\3", path.stem)
        LOGGER.warning(f"Ultralytics YOLO P6 models now use -p6 suffix. Renaming {path.stem} to {new_stem}.")
        path = path.with_name(new_stem + path.suffix)

    unified_path = re.sub(r"(\d+)([nslmx])(.+)?$", r"\1\3", str(path))  # i.e. yolov8x.yaml -> yolov8.yaml
    yaml_file = check_yaml(unified_path, hard=False) or check_yaml(path)
    d = YAML(yaml_file)  # model dict
    d["scale"] = guess_model_scale(path)
    d["yaml_file"] = str(path)
    return d


In [8]:

def parse_model(d, ch, verbose=True):
    """
    Parse a YOLO model.yaml dictionary into a PyTorch model.

    Args:
        d (dict): Model dictionary.
        ch (int): Input channels.
        verbose (bool): Whether to print model details.

    Returns:
        model (torch.nn.Sequential): PyTorch model.
        save (list): Sorted list of output layers.
    """
    import ast

    # Args
    legacy = True  # backward compatibility for v3/v5/v8/v9 models
    max_channels = float("inf")
    nc, act, scales = (d.get(x) for x in ("nc", "activation", "scales"))
    depth, width, kpt_shape = (d.get(x, 1.0) for x in ("depth_multiple", "width_multiple", "kpt_shape"))
    if scales:
        scale = d.get("scale")
        if not scale:
            scale = tuple(scales.keys())[0]
            LOGGER.warning(f"no model scale passed. Assuming scale='{scale}'.")
        depth, width, max_channels = scales[scale]

    if act:
        Conv.default_act = eval(act)  # redefine default activation, i.e. Conv.default_act = torch.nn.SiLU()
        if verbose:
            LOGGER.info(f"{colorstr('activation:')} {act}")  # print

    if verbose:
        LOGGER.info(f"\n{'':>3}{'from':>20}{'n':>3}{'params':>10}  {'module':<45}{'arguments':<30}")
    ch = [ch]
    layers, save, c2 = [], [], ch[-1]  # layers, savelist, ch out
    base_modules = frozenset(
        {
            Classify,
            Conv,
            ConvTranspose,
            GhostConv,
            Bottleneck,
            GhostBottleneck,
            SPP,
            SPPF,
            C2fPSA,
            C2PSA,
            DWConv,
            Focus,
            BottleneckCSP,
            C1,
            C2,
            C2f,
            C3k2,
            RepNCSPELAN4,
            ELAN1,
            ADown,
            AConv,
            SPPELAN,
            C2fAttn,
            C3,
            C3TR,
            C3Ghost,
            # torch.nn.ConvTranspose2d,
            DWConvTranspose2d,
            C3x,
            RepC3,
            PSA,
            SCDown,
            C2fCIB,
            # A2C2f,
        }
    )
    repeat_modules = frozenset(  # modules with 'repeat' arguments
        {
            BottleneckCSP,
            C1,
            C2,
            C2f,
            C3k2,
            C2fAttn,
            C3,
            C3TR,
            C3Ghost,
            C3x,
            RepC3,
            C2fPSA,
            C2fCIB,
            C2PSA,
            # A2C2f,
        }
    )
    for i, (f, n, m, args) in enumerate(d["backbone"] + d["head"]):  # from, number, module, args
        m = (
            getattr(torch.nn, m[3:])
            if "nn." in m
            else getattr(__import__("torchvision").ops, m[16:])
            if "torchvision.ops." in m
            else globals()[m]
        )  # get module
        for j, a in enumerate(args):
            if isinstance(a, str):
                with contextlib.suppress(ValueError):
                    args[j] = locals()[a] if a in locals() else ast.literal_eval(a)
        n = n_ = max(round(n * depth), 1) if n > 1 else n  # depth gain
        if m in base_modules:
            c1, c2 = ch[f], args[0]
            if c2 != nc:  # if c2 not equal to number of classes (i.e. for Classify() output)
                c2 = make_divisible(min(c2, max_channels) * width, 8)
            if m is C2fAttn:  # set 1) embed channels and 2) num heads
                args[1] = make_divisible(min(args[1], max_channels // 2) * width, 8)
                args[2] = int(max(round(min(args[2], max_channels // 2 // 32)) * width, 1) if args[2] > 1 else args[2])

            args = [c1, c2, *args[1:]]
            if m in repeat_modules:
                args.insert(2, n)  # number of repeats
                n = 1
            if m is C3k2:  # for M/L/X sizes
                legacy = False
                if scale in "mlx":
                    args[3] = True
            # if m is A2C2f:
            #     legacy = False
            #     if scale in "lx":  # for L/X sizes
            #         args.extend((True, 1.2))
            if m is C2fCIB:
                legacy = False
        elif m is AIFI:
            args = [ch[f], *args]
        elif m in frozenset({HGStem, HGBlock}):
            c1, cm, c2 = ch[f], args[0], args[1]
            args = [c1, cm, c2, *args[2:]]
            if m is HGBlock:
                args.insert(4, n)  # number of repeats
                n = 1
        elif m is ResNetLayer:
            c2 = args[1] if args[3] else args[1] * 4
        elif m is torch.nn.BatchNorm2d:
            args = [ch[f]]
        elif m is Concat:
            c2 = sum(ch[x] for x in f)
        # elif m in frozenset(
        #     {Detect, WorldDetect, YOLOEDetect, Segment, YOLOESegment, Pose, OBB, ImagePoolingAttn, v10Detect}
        # ):
        #     args.append([ch[x] for x in f])
        #     if m is Segment or m is YOLOESegment:
        #         args[2] = make_divisible(min(args[2], max_channels) * width, 8)
        #     if m in {Detect, YOLOEDetect, Segment, YOLOESegment, Pose, OBB}:
        #         m.legacy = legacy
        elif m in frozenset({Detect,Head}):
            args.append([ch[x] for x in f])
            if m in {Detect,Head}:
                m.legacy = legacy
        elif m is RTDETRDecoder:  # special case, channels arg must be passed in index 1
            args.insert(1, [ch[x] for x in f])
        elif m is CBLinear:
            c2 = args[0]
            c1 = ch[f]
            args = [c1, c2, *args[1:]]
        elif m is CBFuse:
            c2 = ch[f[-1]]
        # elif m in frozenset({TorchVision, Index}):
        #     c2 = args[0]
        #     c1 = ch[f]
        #     args = [*args[1:]]
        else:
            c2 = ch[f]
        m_ = torch.nn.Sequential(*(m(*args) for _ in range(n))) if n > 1 else m(*args)  # module
        t = str(m)[8:-2].replace("__main__.", "")  # module type
        m_.np = sum(x.numel() for x in m_.parameters())  # number params
        m_.i, m_.f, m_.type = i, f, t  # attach index, 'from' index, type
        if verbose:
            LOGGER.info(f"{i:>3}{str(f):>20}{n_:>3}{m_.np:10.0f}  {t:<45}{str(args):<30}")  # print
        save.extend(x % i for x in ([f] if isinstance(f, int) else f) if x != -1)  # append to savelist
        layers.append(m_)
        if i == 0:
            ch = []
        ch.append(c2)
    return torch.nn.Sequential(*layers), sorted(save)



In [9]:
# model_yaml= "my_model.yaml"
# mod_ya = model_yaml if isinstance(model_yaml, dict) else yaml_model_load(model_yaml)

In [10]:
# nc = 80  # replace with your actual number of classes
# ch = 3
# yaml_dict = mod_ya.copy()
# yaml_dict['backbone'] = [
#     [-1, 1, 'Conv', [64]],  # 0
#     [-1, 1, 'Conv', [128]], # 1
#     [-1, 1, 'Conv', [256]]  # 2
# ]

# yaml_dict['head'] = [
#     [[0, 1, 2], 1, 'Detect', [nc]]  # 3 inputs for Detect
# ]
# # Step 3: Parse it into a model
# model, _ = parse_model(deepcopy(yaml_dict), ch=ch, verbose=True)
# print("\n\nParsed Model:\n", model)

In [11]:
ch=3
verbose = True
model_yaml= "my_model.yaml"
mod_ya = model_yaml if isinstance(model_yaml, dict) else yaml_model_load(model_yaml)
# model, save = parse_model(deepcopy(mod_ya), ch=ch, verbose=verbose)
from ultralytics.nn.tasks import parse_model
model, save  = parse_model(deepcopy(mod_ya), ch=ch, verbose=verbose)


no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.model_classes.Conv            [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.model_classes.Conv            [16, 32, 3, 2]                


  2                  -1  1      7360  ultralytics.nn.model_classes.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.model_classes.Conv            [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.model_classes.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.model_classes.Conv            [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.model_classes.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.model_classes.Conv            [128, 256, 3, 2]              
  8                  -1  1    460288  ultralytics.nn.model_classes.C2f             [256, 256, 1, True]           
  9                  -1  1    164608  ultralytics.nn.model_classes.SPPF            [256, 256, 5]                 
 10                  -1  1         0  ultralytics.nn.model_classes.Upsample        [2, '

In [19]:

from roboflow import Roboflow
rf = Roboflow(api_key="u8Ur5ywKLswqwt0uMTXl")
project = rf.workspace("poacher-ifyuf").project("poacher")
version = project.version(2)
path = r"C:\Users\KIIT\OneDrive\Desktop\Vedant_Official\vedant projects and works\ML_Deep_learning_projects\Deep_Learning_Projects\Project_1\Dataset"
dataset = version.download("yolov8", location=path)

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.134 is required but found version=8.3.28, to fix: `pip install ultralytics==8.0.134`



Extracting Dataset Version Zip to C:\Users\KIIT\OneDrive\Desktop\Vedant_Official\vedant projects and works\ML_Deep_learning_projects\Deep_Learning_Projects\Project_1\Dataset in yolov8::   1%|          | 12/1396 [00:00<00:01, 819.68it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\KIIT\\OneDrive\\Desktop\\Vedant_Official\\vedant projects and works\\ML_Deep_learning_projects\\Deep_Learning_Projects\\Project_1\\Dataset\\test\\images\\Poachers20with20gun20prowling20in20Bannerghatta-1631912935-1587208022JPG_jpg.rf.21b5c4254f11de9f1bee87b699100d75.jpg'

In [ ]:
from ultralytics import YOLO
import argparse

def train(model_yaml, data_yaml, epochs=100, imgsz=640, batch=16, device='0'):
    # Load model from architecture YAML
    model = YOLO(model_yaml)

    # Start training
    model.train(
        data=data_yaml,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        device=device,
        workers=8,
        name="custom-yolo-train"
    )

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--model", type=str, default="Architecture\model_architecture\my_model.yaml", help="Path to custom model YAML")
    parser.add_argument("--data", type=str, default="data.yaml", help="Path to dataset config YAML")
    parser.add_argument("--epochs", type=int, default=100)
    parser.add_argument("--imgsz", type=int, default=640)
    parser.add_argument("--batch", type=int, default=16)
    parser.add_argument("--device", type=str, default='0')

    args = parser.parse_args()
    train(
        model_yaml=args.model,
        data_yaml=args.data,
        epochs=args.epochs,
        imgsz=args.imgsz,
        batch=args.batch,
        device=args.device
    )
